## Install

In [1]:
# install MIM
!pip install -U openmim
!mim install mmengine
!mim install 'mmcv>=2.0.0rc1'
!mim install 'mmdet>=3.0.0rc0'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 3.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.1/372.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 kB 22.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 MB 5.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━

In [2]:
# install MMOCR
# !git clone https://github.com/open-mmlab/mmocr.git
!git clone https://github.com/yellowjs0304/mmocr.git
%cd mmocr
!pip install -v -e .
# "-v" increases pip's verbosity.
# "-e" means installing the project in editable mode,
# That is, any local modifications on the code will take effect immediately.

Cloning into 'mmocr'...
remote: Enumerating objects: 16277, done.
remote: Counting objects: 100% (188/188), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 16277 (delta 56), reused 138 (delta 42), pack-reused 16089
Receiving objects: 100% (16277/16277), 16.24 MiB | 21.26 MiB/s, done.
Resolving deltas: 100% (10654/10654), done.
/content/mmocr
Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mmocr
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-g5n9a591/mmocr.egg-info
  writing /tmp/pip-pip-egg-info-g5n9a591/mmocr.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-g5n9a591/mmocr.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-g5n9a591/mmocr.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-g5n9a

In [3]:
from mmocr.datasets import OCRDataset
from mmengine.registry import init_default_scope
from mmocr.apis import MMOCRInferencer
from mmengine.runner import Runner
import time
import mmcv
import matplotlib.pyplot as plt 

## Data Load

In [4]:
# %cd /content
# !pip install kaggle

# from google.colab import files
# files.upload()

# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

# !kaggle datasets download -d jinn0135/temp-sign
# !unzip -q temp_sign.zip -d temp_sign/
# root = '/content/temp_sign/Temp/'

In [5]:
from google.colab import drive
drive.mount('/content/drive')
root = '/content/drive/MyDrive/Colab Notebooks/pj4_desktop/Temp/'

Mounted at /content/drive


In [6]:
init_default_scope('mmocr')
pipeline = [dict(type='LoadImageFromFile'), 
            dict(type='LoadOCRAnnotations', 
                with_polygon=True, 
                with_bbox=True, 
                with_label=True,
                ), 
            dict(type='Resize', scale=(640, 640), keep_ratio=True), 
            dict(type='Pad', size=(640, 640)), 
            dict(type='PackTextDetInputs', 
                meta_keys=('img_path', 'ori_shape', 'img_shape'))
            ]
trainset = OCRDataset(data_root=root+'temp_Sign', 
                      ann_file=root+'temp_train_annotation.json', 
                      test_mode=False, 
                      pipeline=pipeline)
validset = OCRDataset(data_root=root+'temp_Sign', 
                      ann_file=root+'temp_valid_annotation.json', 
                      test_mode=True, 
                      pipeline=pipeline)
testset = OCRDataset(data_root=root+'temp_Sign', 
                      ann_file=root+'temp_test_annotation.json', 
                      test_mode=True, 
                      pipeline=pipeline)                      

In [7]:
batch_s = 16
trainloader = dict(batch_size=batch_s,
                   num_workers=0,
                   persistent_workers=True,
                   sampler=dict(type='DefaultSampler', shuffle=True),
                   dataset=trainset)
validloader = dict(batch_size=batch_s,
                   num_workers=0,
                   persistent_workers=True,
                   sampler=dict(type='DefaultSampler', shuffle=True),
                   dataset=validset)
testloader = dict(batch_size=batch_s,
                   num_workers=0,
                   persistent_workers=True,
                   sampler=dict(type='DefaultSampler', shuffle=True),
                   dataset=testset)